In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from copy import deepcopy
from datetime import datetime
from utils.elasticsearch_utils import ElasticSearchUtils

/home/workspace/data-center/utils/elasticsearch_utils.py:21: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
def dump_data(index):
    """ """
    host_info = {
        'host': 'https://corpus.ncsoft.com:9200',
        'http_auth': 'elastic:nlplab',
    }

    utils = ElasticSearchUtils(**host_info)

    query = {
        "query": {
            "bool": {
                "must": {
                    "match": {
                        'group_name': '잠 안오는 사람들의 모임'
                    }
                }
            }
        }
    }

    doc_list = []
    utils.export(index=index, result=doc_list, query=query)

    return doc_list


def flatten(doc_list, column_list):
    """ """
    result = []
    for doc in doc_list:
        if 'reply_list' not in doc or len(doc['reply_list']) == 0:
            continue

        for reply in doc['reply_list']:
            item = {
                'category': doc['category'],
                'group_name': doc['group_name'],
                'curl_date': doc['curl_date'],
                'document_id': doc['document_id'],
                'content': doc['content'].strip(),

                'user_name': reply['user_name'],
                'reply_to': reply['reply_to'],
                'text': reply['text'].strip(),
            }
            
            if item['text'] != '':
                result.append(item)

            if 'reply_list' not in reply:
                continue

            for reply_to in reply['reply_list']:
                item = {
                    'category': doc['category'],
                    'group_name': doc['group_name'],
                    'curl_date': doc['curl_date'],
                    'document_id': doc['document_id'],
                    'content': doc['content'].strip(),

                    'user_name': reply_to['user_name'],
                    'reply_to': reply_to['reply_to'],
                    'text': reply_to['text'].strip(),
                }
                
                if item['text'] != '':
                    result.append(item)

    return result

In [4]:
index = 'crawler-facebook'

doc_list = dump_data(index=index)

print('{:,}'.format(len(doc_list)))


8,190


In [5]:
column_list = 'category group_name curl_date document_id reply_to user_name content text reply_list'.split(' ')

doc2_list = flatten(doc_list=doc_list, column_list=column_list)

len(doc2_list)

167425

In [6]:
column_list = 'category group_name curl_date document_id reply_to user_name content text'.split(' ')

df = pd.DataFrame(doc2_list)

df.fillna('', inplace=True)

df = df[column_list]
df = df[ df['group_name'] == '잠 안오는 사람들의 모임' ]
    
print('{:,}'.format(len(df)))
df.head()

167,393


,category,group_name,curl_date,document_id,reply_to,user_name,content,text
0,친목,잠 안오는 사람들의 모임,2020-01-06T15:41:44.288062+09:00,562128621033889-565786914001393,,이현수,자야되는데 집 앞 공사중~ 오늘도 강제기상\n ....,저는 등기우편 왔네요ㅜ
1,친목,잠 안오는 사람들의 모임,2020-01-06T15:41:44.288062+09:00,562128621033889-565786914001393,이현수,선희,자야되는데 집 앞 공사중~ 오늘도 강제기상\n ....,엥
2,친목,잠 안오는 사람들의 모임,2020-01-06T15:41:44.288062+09:00,562128621033889-565786914001393,선희,이현수,자야되는데 집 앞 공사중~ 오늘도 강제기상\n ....,다른우편이면 그냥 우편함에 넣는데 얘는 서명해야해서 강제기상이에요 겨우잠들었는데
3,친목,잠 안오는 사람들의 모임,2020-01-06T15:41:44.288062+09:00,562128621033889-565786914001393,,서진석,자야되는데 집 앞 공사중~ 오늘도 강제기상\n ....,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\nㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
4,친목,잠 안오는 사람들의 모임,2020-01-06T15:41:44.288062+09:00,562128621033889-565786914001393,,서진석,자야되는데 집 앞 공사중~ 오늘도 강제기상\n ....,역시 가는날이 장날이네욬ㅋㅋㅋㅋ


In [7]:
df[df['document_id'] == '562128621033889-565786914001393']

,category,group_name,curl_date,document_id,reply_to,user_name,content,text
0,친목,잠 안오는 사람들의 모임,2020-01-06T15:41:44.288062+09:00,562128621033889-565786914001393,,이현수,자야되는데 집 앞 공사중~ 오늘도 강제기상\n ....,저는 등기우편 왔네요ㅜ
1,친목,잠 안오는 사람들의 모임,2020-01-06T15:41:44.288062+09:00,562128621033889-565786914001393,이현수,선희,자야되는데 집 앞 공사중~ 오늘도 강제기상\n ....,엥
2,친목,잠 안오는 사람들의 모임,2020-01-06T15:41:44.288062+09:00,562128621033889-565786914001393,선희,이현수,자야되는데 집 앞 공사중~ 오늘도 강제기상\n ....,다른우편이면 그냥 우편함에 넣는데 얘는 서명해야해서 강제기상이에요 겨우잠들었는데
3,친목,잠 안오는 사람들의 모임,2020-01-06T15:41:44.288062+09:00,562128621033889-565786914001393,,서진석,자야되는데 집 앞 공사중~ 오늘도 강제기상\n ....,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\nㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
4,친목,잠 안오는 사람들의 모임,2020-01-06T15:41:44.288062+09:00,562128621033889-565786914001393,,서진석,자야되는데 집 앞 공사중~ 오늘도 강제기상\n ....,역시 가는날이 장날이네욬ㅋㅋㅋㅋ
5,친목,잠 안오는 사람들의 모임,2020-01-06T15:41:44.288062+09:00,562128621033889-565786914001393,,선희,자야되는데 집 앞 공사중~ 오늘도 강제기상\n ....,ㅋㅋㅋㅋㅋㅋㅋ아
6,친목,잠 안오는 사람들의 모임,2020-01-06T15:41:44.288062+09:00,562128621033889-565786914001393,,박병규,자야되는데 집 앞 공사중~ 오늘도 강제기상\n ....,요새 잠을 좀 자기는 하셨나요? ㅋㅋㅋㅋ
7,친목,잠 안오는 사람들의 모임,2020-01-06T15:41:44.288062+09:00,562128621033889-565786914001393,박병규,선희,자야되는데 집 앞 공사중~ 오늘도 강제기상\n ....,그나마..?
8,친목,잠 안오는 사람들의 모임,2020-01-06T15:41:44.288062+09:00,562128621033889-565786914001393,,이혁진,자야되는데 집 앞 공사중~ 오늘도 강제기상\n ....,이어플러그 사용해보세요ㅠㅠ


In [ ]:
df[['content', 'text']]

In [8]:
dt = datetime.now().strftime('%Y-%m-%d')

df.to_excel('data/{}-{}.xlsx'.format(index, dt), index=False)